# Reviewing new EPMC evidence


For more details on the scope of this ticket, see: [#1867](https://github.com/opentargets/platform/issues/1867)


### Fetching epmc evidence data from google buckets:

```bash
TARGET_DIR='/Users/dsuveges/project_data/epmc_evidence'

# Fetching old evidence:
gsutil cp -r gs://otar000-evidence_input/EPMC/json/epmc-2021-11-08.json.gz  ${TARGET_DIR}

# Fetching new evidence:
gsutil cp -r gs://ot-snapshots/literature/20210920/new_model/evidence ${TARGET_DIR}/

# Renaming the new evidence file:
mv ${TARGET_DIR}/evidence ${TARGET_DIR}/epmc_new_evid_2021.10.25.parquet
```

In [1]:
from pyspark.sql.functions import (
    col, udf, struct, lit, split, regexp_replace, create_map, min as spark_min, max as spark_max,
    count
)
from pyspark.sql.types import FloatType, ArrayType, StructType, StructField
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from itertools import chain


# for heatmap:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

# establish spark connection
sparkConf = (
    SparkConf()
    .set('spark.driver.memory', '15g')
    .set('spark.executor.memory', '15g')
    .set('spark.driver.maxResultSize', '0')
)
spark = (
    SparkSession.builder
    .config(conf=sparkConf)
    .master('local[*]')
    .getOrCreate()
)

In [ ]:
# Reading new dataset:
new_data = (
    spark.read.parquet('/Users/dsuveges/project_data/epmc_evidence/epmc_new_evid_2021.10.25.parquet')
    .drop('datasourceId', 'datatypeId', '')
    .persist()
) 
old_data = (
    spark.read.json('/Users/dsuveges/project_data/epmc_evidence/epmc-2021-11-08.json.gz')
    .drop('datasourceId', 'datatypeId')
    .persist()
)

In [8]:
(
    spark.read.json('/Users/dsuveges/project_data/epmc_evidence/epmc-2021-11-08.json.gz')
    .drop('datasourceId', 'datatypeId')
    .persist()
) 

root
 |-- diseaseFromSourceMappedId: string (nullable = true)
 |-- literature: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pmcIds: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- resourceScore: double (nullable = true)
 |-- targetFromSourceId: string (nullable = true)
 |-- textMiningSentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- dEnd: long (nullable = true)
 |    |    |-- dStart: long (nullable = true)
 |    |    |-- section: string (nullable = true)
 |    |    |-- tEnd: long (nullable = true)
 |    |    |-- tStart: long (nullable = true)
 |    |    |-- text: string (nullable = true)

